In [40]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
from pandas import json_normalize

from geopy.geocoders import Nominatim

import requests

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

from bs4 import BeautifulSoup

import xml

import folium

In [9]:
url = requests.get('https://en.wikipedia.org/wiki/Community_areas_in_Chicago').text
soup = BeautifulSoup(url,'lxml')

In [10]:
table_contents=[]
table=soup.find('table')
i = 1
cell = {}
#print(table)
for row in table.findAll('th'):
    if (i > 7 and len(table_contents) < 77):
        cell['Community Area'] = row.text.strip()
        table_contents.append(cell)
        cell = {}
    i +=1

df=pd.DataFrame(table_contents)

In [11]:
df

,Community Area
0,Rogers Park
1,West Ridge
2,Uptown
3,Lincoln Square
4,North Center
5,Lake View
6,Lincoln Park
7,Near North Side
8,Edison Park
9,Norwood Park


In [12]:
df['Community Area']=df['Community Area'].replace({"(The) Loop[11]": "The Loop"})
df

,Community Area
0,Rogers Park
1,West Ridge
2,Uptown
3,Lincoln Square
4,North Center
5,Lake View
6,Lincoln Park
7,Near North Side
8,Edison Park
9,Norwood Park


In [13]:
df['Latitude'] = ""
df['Longitude'] = ""
df

,Community Area,Latitude,Longitude
0,Rogers Park,,
1,West Ridge,,
2,Uptown,,
3,Lincoln Square,,
4,North Center,,
5,Lake View,,
6,Lincoln Park,,
7,Near North Side,,
8,Edison Park,,
9,Norwood Park,,


In [14]:
df.shape

(77, 3)

In [15]:
geolocator = Nominatim(user_agent="ny_explorer")
for index, row in df.iterrows():
    address = row['Community Area'] + ', Chicago'
    try:
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        df.loc[index, 'Latitude'] = latitude
        df.loc[index, 'Longitude'] = longitude
        print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
    except AttributeError:
        print('Cannot find coordinates for: {}, will drop index: {}'.format(address, index))

The geograpical coordinate of Rogers Park, Chicago are 42.0090367, -87.6768492.
The geograpical coordinate of West Ridge, Chicago are 42.0035482, -87.6962426.
The geograpical coordinate of Uptown, Chicago are 41.9666299, -87.6555458.
The geograpical coordinate of Lincoln Square, Chicago are 41.9685116, -87.6886528.
The geograpical coordinate of North Center, Chicago are 41.9561073, -87.6791596.
The geograpical coordinate of Lake View, Chicago are 41.9439185, -87.6540759.
The geograpical coordinate of Lincoln Park, Chicago are 41.9399447, -87.63611961353928.
The geograpical coordinate of Near North Side, Chicago are 41.9000327, -87.6344975.
The geograpical coordinate of Edison Park, Chicago are 42.0043532, -87.8128346.
The geograpical coordinate of Norwood Park, Chicago are 41.9880657, -87.8027487.
The geograpical coordinate of Jefferson Park, Chicago are 41.9697375, -87.7631179.
The geograpical coordinate of Forest Glen, Chicago are 41.9854932, -87.754681.
The geograpical coordinate of

In [16]:
df['Latitude'].replace('', np.nan, inplace=True)
df.dropna(subset=['Latitude'], inplace=True)
df

,Community Area,Latitude,Longitude
0,Rogers Park,42.009037,-87.6768
1,West Ridge,42.003548,-87.6962
2,Uptown,41.966630,-87.6555
3,Lincoln Square,41.968512,-87.6887
4,North Center,41.956107,-87.6792
5,Lake View,41.943919,-87.6541
6,Lincoln Park,41.939945,-87.6361
7,Near North Side,41.900033,-87.6345
8,Edison Park,42.004353,-87.8128
9,Norwood Park,41.988066,-87.8027


In [17]:
df.shape

(77, 3)

In [18]:
map_chicago = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, area in zip(df['Latitude'], df['Longitude'], df['Community Area']):
    label = '{}'.format(area)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chicago)  
    
map_chicago

In [19]:
CLIENT_ID = 'QRAB2UWUYAEIFYYDYJWCVZD4PE1UA3KDW1QPNWPZL5XBZMTF' # your Foursquare ID
CLIENT_SECRET = 'EDLRVP5RIFHNJQOVO2S3S035R00UARTH2DJ00DXTC35HTPLS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: QRAB2UWUYAEIFYYDYJWCVZD4PE1UA3KDW1QPNWPZL5XBZMTF
CLIENT_SECRET:EDLRVP5RIFHNJQOVO2S3S035R00UARTH2DJ00DXTC35HTPLS


In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Community Area', 
                  'Community Area Latitude', 
                  'Community Area Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
chicago_venues = getNearbyVenues(names=df['Community Area'],latitudes=df['Latitude'],longitudes=df['Longitude'])
chicago_venues

,Community Area,Community Area Latitude,Community Area Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rogers Park,42.009037,-87.676849,Bark Place,42.010080,-87.675223,Pet Store
1,Rogers Park,42.009037,-87.676849,El Famous Burrito,42.010421,-87.674204,Mexican Restaurant
2,Rogers Park,42.009037,-87.676849,Taqueria & Restaurant Cd. Hidalgo,42.011634,-87.674484,Mexican Restaurant
3,Rogers Park,42.009037,-87.676849,Tamales Lo Mejor De Guerrero,42.009784,-87.674186,Mexican Restaurant
4,Rogers Park,42.009037,-87.676849,Nueva Italy Pizzeria,42.011629,-87.674205,Pizza Place
5,Rogers Park,42.009037,-87.676849,Panaderia Ayutla,42.008405,-87.673536,Bakery
6,Rogers Park,42.009037,-87.676849,A & T Pancake House and Grill,42.009969,-87.674442,American Restaurant
7,Rogers Park,42.009037,-87.676849,Romanian Kosher Sausage Co.,42.012765,-87.674692,Deli / Bodega
8,Rogers Park,42.009037,-87.676849,China Hut,42.011516,-87.674499,Chinese Restaurant
9,Rogers Park,42.009037,-87.676849,China Spring,42.009111,-87.673965,Chinese Restaurant


In [22]:
chicago_venues.groupby(['Community Area']).count()

,Community Area Latitude,Community Area Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Community Area,,,,,,
Albany Park,24,24,24,24,24,24
Archer Heights,17,17,17,17,17,17
Armour Square,13,13,13,13,13,13
Ashburn,10,10,10,10,10,10
Auburn Gresham,1,1,1,1,1,1
Austin,12,12,12,12,12,12
Avalon Park,15,15,15,15,15,15
Avondale,29,29,29,29,29,29
Belmont Cragin,17,17,17,17,17,17


In [23]:
print('There are {} uniques categories.'.format(len(chicago_venues['Venue Category'].unique())))

There are 246 uniques categories.


In [24]:
chicago_onehot = pd.get_dummies(chicago_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
chicago_onehot['Community Area'] = chicago_venues['Community Area'] 

# move neighborhood column to the first column
neigh_col = chicago_onehot['Community Area']
chicago_onehot.drop(labels=['Community Area'], axis=1,inplace = True)
chicago_onehot.insert(0, 'Community Area', neigh_col)
#fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
#toronto_onehot = toronto_onehot[fixed_columns]

chicago_onehot.head()

,Community Area,ATM,Accessories Store,Adult Boutique,African Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,College Cafeteria,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Currency Exchange,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Eye Doctor,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hostel,Hot Dog Joint,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Light Rail Station,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,National Park,Nature Preserve,New American Restaurant,Nightclub,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Record Shop,Recording Studio,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Ukrainian Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Vineyard,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Rogers Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [25]:
chicago_onehot.shape

(1508, 247)

In [26]:
chicago_grouped = chicago_onehot.groupby('Community Area').mean().reset_index()
chicago_grouped.head()

,Community Area,ATM,Accessories Store,Adult Boutique,African Restaurant,Airport,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,College Cafeteria,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Currency Exchange,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Eye Doctor,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hostel,Hot Dog Joint,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Light Rail Station,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,National Park,Nature Preserve,New American Restaurant,Nightclub,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Record Shop,Recording Studio,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Ukrainian Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Vineyard,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Albany Park,0.0,0.041667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.041667,0.0,0.0,0.0,0.0,0.041667,0.0,0.041667,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.041667,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.083333,0.0,0.0,0.0,0.041667,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.041667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.04

In [27]:
chicago_grouped.shape

(76, 247)

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
#create new dataframe with the top 5 venues
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Community Area']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Community Area'] = chicago_grouped['Community Area']

for ind in np.arange(chicago_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(chicago_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Community Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Albany Park,Grocery Store,Mexican Restaurant,Chinese Restaurant,Asian Restaurant,Mobile Phone Shop
1,Archer Heights,Mexican Restaurant,Mobile Phone Shop,Grocery Store,Optical Shop,Bank
2,Armour Square,Chinese Restaurant,Cosmetics Shop,Business Service,Grocery Store,Asian Restaurant
3,Ashburn,Automotive Shop,Gift Shop,Snack Place,Martial Arts School,Bus Station
4,Auburn Gresham,Discount Store,Yoga Studio,Ethiopian Restaurant,Food,Flower Shop


In [51]:
kclusters = 5

chicago_grouped_clustering = chicago_grouped.drop('Community Area', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(chicago_grouped_clustering)

kmeans.labels_[0:10]

array([3, 3, 3, 3, 4, 2, 3, 3, 3, 3])

In [52]:
neighborhoods_venues_sorted

,Community Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Albany Park,Grocery Store,Mexican Restaurant,Chinese Restaurant,Asian Restaurant,Mobile Phone Shop
1,Archer Heights,Mexican Restaurant,Mobile Phone Shop,Grocery Store,Optical Shop,Bank
2,Armour Square,Chinese Restaurant,Cosmetics Shop,Business Service,Grocery Store,Asian Restaurant
3,Ashburn,Automotive Shop,Gift Shop,Snack Place,Martial Arts School,Bus Station
4,Auburn Gresham,Discount Store,Yoga Studio,Ethiopian Restaurant,Food,Flower Shop
5,Austin,Park,Bus Station,Intersection,Wings Joint,Donut Shop
6,Avalon Park,Fast Food Restaurant,Business Service,Boutique,Pizza Place,Burger Joint
7,Avondale,Food Truck,Chinese Restaurant,Park,Diner,Supermarket
8,Belmont Cragin,Mexican Restaurant,Grocery Store,Bakery,Nightclub,Thrift / Vintage Store
9,Beverly Hills,Yoga Studio,Video Store,Sandwich Place,Park,Train Station


In [53]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

chicago_merged = df
chicago_merged.columns = ['Community Area', 'Latitude', 'Longitude']

chicago_merged = chicago_merged.join(neighborhoods_venues_sorted.set_index('Community Area'), on = 'Community Area' )

chicago_merged.head() 

,Community Area,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Rogers Park,42.009037,-87.6768,3.0,Mexican Restaurant,Pizza Place,Chinese Restaurant,Sandwich Place,Bakery
1,West Ridge,42.003548,-87.6962,0.0,Convenience Store,Insurance Office,Business Service,Fried Chicken Joint,Yoga Studio
2,Uptown,41.966630,-87.6555,3.0,Coffee Shop,Mexican Restaurant,Pizza Place,Diner,Bus Station
3,Lincoln Square,41.968512,-87.6887,3.0,Bar,Thai Restaurant,Coffee Shop,Pizza Place,Café
4,North Center,41.956107,-87.6792,3.0,Bar,Coffee Shop,Brewery,Video Store,Bus Station


In [54]:
chicago_merged.dropna(subset = ['Cluster Labels'], inplace= True)

In [56]:
chicago_merged['Cluster Labels'] = chicago_merged['Cluster Labels'].astype(int)

In [57]:
chicago_merged.head()

,Community Area,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Rogers Park,42.009037,-87.6768,3,Mexican Restaurant,Pizza Place,Chinese Restaurant,Sandwich Place,Bakery
1,West Ridge,42.003548,-87.6962,0,Convenience Store,Insurance Office,Business Service,Fried Chicken Joint,Yoga Studio
2,Uptown,41.966630,-87.6555,3,Coffee Shop,Mexican Restaurant,Pizza Place,Diner,Bus Station
3,Lincoln Square,41.968512,-87.6887,3,Bar,Thai Restaurant,Coffee Shop,Pizza Place,Café
4,North Center,41.956107,-87.6792,3,Bar,Coffee Shop,Brewery,Video Store,Bus Station


In [63]:
chicago_asian = chicago_grouped[['Community Area','Asian Restaurant']]

In [72]:
chicago_merged = chicago_merged.merge(chicago_asian, on = 'Community Area')

In [90]:
chicago_merged

,Community Area,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Asian Restaurant
0,Rogers Park,42.009037,-87.6768,3,Mexican Restaurant,Pizza Place,Chinese Restaurant,Sandwich Place,Bakery,0.000000
1,West Ridge,42.003548,-87.6962,0,Convenience Store,Insurance Office,Business Service,Fried Chicken Joint,Yoga Studio,0.000000
2,Uptown,41.966630,-87.6555,3,Coffee Shop,Mexican Restaurant,Pizza Place,Diner,Bus Station,0.013699
3,Lincoln Square,41.968512,-87.6887,3,Bar,Thai Restaurant,Coffee Shop,Pizza Place,Café,0.000000
4,North Center,41.956107,-87.6792,3,Bar,Coffee Shop,Brewery,Video Store,Bus Station,0.000000
5,Lake View,41.943919,-87.6541,3,Gay Bar,Sandwich Place,Mexican Restaurant,Sports Bar,Pizza Place,0.011905
6,Lincoln Park,41.939945,-87.6361,3,Harbor / Marina,Boat or Ferry,Bus Station,Juice Bar,Park,0.000000
7,Near North Side,41.900033,-87.6345,3,Yoga Studio,Gym / Fitness Center,Gym,Park,Sandwich Place,0.000000
8,Edison Park,42.004353,-87.8128,3,Bar,Italian Restaurant,Mexican Restaurant,Theater,Restaurant,0.000000
9,Norwood Park,41.988066,-87.8027,3,Park,Diner,Hobby Shop,Dog Run,Clothing Store,0.000000


In [74]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(chicago_merged['Latitude'], chicago_merged['Longitude'], chicago_merged['Community Area'], chicago_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

['#8000ff', '#00b5eb', '#80ffb4', '#ffb360', '#ff0000']


In [85]:
chicago_merged.loc[chicago_merged['Cluster Labels'] == 0, chicago_merged.columns[[0] + list(range(4, chicago_merged.shape[1]))]]

,Community Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Asian Restaurant
1,West Ridge,Convenience Store,Insurance Office,Business Service,Fried Chicken Joint,Yoga Studio,0.0
28,North Lawndale,Food,BBQ Joint,Convenience Store,Concert Hall,Fast Food Restaurant,0.0
51,East Side,Food,Construction & Landscaping,Juice Bar,Trail,Video Store,0.0
59,New City,American Restaurant,Construction & Landscaping,Athletics & Sports,Department Store,Optical Shop,0.0
64,Chicago Lawn,Pizza Place,Fast Food Restaurant,Mexican Restaurant,American Restaurant,Fish & Chips Shop,0.0
67,Greater Grand Crossing,Fried Chicken Joint,American Restaurant,Pizza Place,Art Gallery,Liquor Store,0.0
71,Washington Heights,Caribbean Restaurant,Breakfast Spot,Pizza Place,Snack Place,Bus Station,0.0


In [86]:
chicago_merged.loc[chicago_merged['Cluster Labels'] == 1, chicago_merged.columns[[0] + list(range(4, chicago_merged.shape[1]))]]

,Community Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Asian Restaurant
74,O'Hare,Pool,Yoga Studio,Electronics Store,Flower Shop,Flea Market,0.0


In [87]:
chicago_merged.loc[chicago_merged['Cluster Labels'] == 2, chicago_merged.columns[[0] + list(range(4, chicago_merged.shape[1]))]]

,Community Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Asian Restaurant
16,Dunning,Construction & Landscaping,Deli / Bodega,Intersection,Bar,Yoga Studio,0.0
24,Austin,Park,Bus Station,Intersection,Wings Joint,Donut Shop,0.0
35,Oakland,Park,Public Art,Eastern European Restaurant,Flower Shop,Flea Market,0.0
41,Woodlawn,Fast Food Restaurant,Coffee Shop,Bus Station,Park,Yoga Studio,0.0
42,South Shore,Bus Station,Multiplex,Record Shop,Deli / Bodega,Baseball Field,0.0
43,Chatham,Boutique,Park,Fast Food Restaurant,Ice Cream Shop,Donut Shop,0.0
46,Burnside,Home Service,Train Station,Intersection,Bus Station,Yoga Studio,0.0
47,Calumet Heights,Park,Deli / Bodega,Gym / Fitness Center,Bus Station,Yoga Studio,0.0
48,Roseland,Food,Intersection,Yoga Studio,Electronics Store,Flower Shop,0.0
72,Mount Greenwood,Cosmetics Shop,Vineyard,Intersection,Park,Home Service,0.0


In [88]:
chicago_merged.loc[chicago_merged['Cluster Labels'] == 3, chicago_merged.columns[[0] + list(range(4, chicago_merged.shape[1]))]]

,Community Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Asian Restaurant
0,Rogers Park,Mexican Restaurant,Pizza Place,Chinese Restaurant,Sandwich Place,Bakery,0.000000
2,Uptown,Coffee Shop,Mexican Restaurant,Pizza Place,Diner,Bus Station,0.013699
3,Lincoln Square,Bar,Thai Restaurant,Coffee Shop,Pizza Place,Café,0.000000
4,North Center,Bar,Coffee Shop,Brewery,Video Store,Bus Station,0.000000
5,Lake View,Gay Bar,Sandwich Place,Mexican Restaurant,Sports Bar,Pizza Place,0.011905
6,Lincoln Park,Harbor / Marina,Boat or Ferry,Bus Station,Juice Bar,Park,0.000000
7,Near North Side,Yoga Studio,Gym / Fitness Center,Gym,Park,Sandwich Place,0.000000
8,Edison Park,Bar,Italian Restaurant,Mexican Restaurant,Theater,Restaurant,0.000000
9,Norwood Park,Park,Diner,Hobby Shop,Dog Run,Clothing Store,0.000000
10,Jefferson Park,Bar,Ice Cream Shop,Coffee Shop,Pharmacy,Chinese Restaurant,0.000000


In [89]:
chicago_merged.loc[chicago_merged['Cluster Labels'] == 4, chicago_merged.columns[[0] + list(range(4, chicago_merged.shape[1]))]]

,Community Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Asian Restaurant
69,Auburn Gresham,Discount Store,Yoga Studio,Ethiopian Restaurant,Food,Flower Shop,0.0
